In [64]:
import pandas as pd
from pandas import DataFrame
import sodapy as sd
import matplotlib as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas_bokeh as pbk
pbk.output_notebook()
import geopandas
import seaborn as sns

Loading BokehJS ...

In [65]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

from sodapy import Socrata

client = Socrata("analisi.transparenciacatalunya.cat", None)

results = client.get("rmgc-ncpb", limit=16774)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [66]:
# Tenim en la columnna 'dat' el dia i l'hora juntes, volem separar-les en dues noves columnes 'data' i 'hora'
dist_morts=df[["dat","f_morts"]]
# Canviem la T que separa dia i hora per identificar la separació dues vegades: amb ' ' i amb '-'
df['dat'] = df['dat'].str.replace('T',' ')
df[['data','hora']] = df["dat"].str.split(" ", 1, expand=True)
df['data']=df['data'].astype("datetime64")
df["hora"] = df["hora"].str.strip(" ")

df[['hores','minuts']] = df["hora"].str.split(":", 1, expand=True)
df["data"].dtypes
df

,any,zona,dat,via,pk,nommun,nomcom,nomdem,f_morts,f_ferits_greus,...,d_vent,grupdialab,hor,gruphor,tipacc,tipdia,data,hora,hores,minuts
0,2010,Zona urbana,2010-01-25 23:33:00.000,SE,999999,CANOVES I SAMALUS,Valles Oriental,Barcelona,0,1,...,"Calma, vent molt suau",Feiners,2333,Nit,Col.lisió de vehicles en marxa,dill-dij,2010-01-25,23:33:00.000,23,33:00.000
1,2010,Carretera,2010-10-31 01:00:00.000,N-240,999,LLEIDA,Segria,Lleida,0,1,...,"Calma, vent molt suau",CapDeSetmana,1,Nit,Sortida de la calcada sense especificar,dg,2010-10-31,01:00:00.000,01,00:00.000
2,2010,Carretera,2010-05-17 15:27:00.000,N-II,7087,FORNELLS DE LA SELVA,Girones,Girona,1,0,...,"Calma, vent molt suau",Feiners,1527,Tarda,Col.lisió de vehicles en marxa,dill-dij,2010-05-17,15:27:00.000,15,27:00.000
3,2010,Zona urbana,2010-08-21 22:30:00.000,SE,999999,BARCELONA,Barcelones,Barcelona,0,2,...,"Calma, vent molt suau",CapDeSetmana,223,Nit,Col.lisió de vehicles en marxa,dis,2010-08-21,22:30:00.000,22,30:00.000
4,2010,Zona urbana,2010-05-07 17:45:00.000,SE,999999,BADALONA,Barcelones,Barcelona,0,1,...,"Calma, vent molt suau",CapDeSetmana,1745,Tarda,Bolcada a la calcada,div,2010-05-07,17:45:00.000,17,45:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16769,2018,Zona urbana,2018-11-20 09:45:00.000,SE,999999,BARCELONA,Barcelones,Barcelona,0,1,...,"Calma, vent molt suau",Feiners,945,Matí,Col.lisió de vehicles en marxa,dill-dij,2018-11-20,09:45:00.000,09,45:00.000
16770,2018,Carretera,2018-12-03 02:01:00.000,AP-7,173,CASTELLVI DE ROSANES,Baix Llobregat,Barcelona,0,1,...,"Calma, vent molt suau",Feiners,201,Nit,Col.lisió de vehicles en marxa,dill-dij,2018-12-03,02:01:00.000,02,01:00.000
16771,2018,Zona urbana,2018-09-30 21:19:00.000,SE,999999,"VENDRELL, EL",Baix Penedes,Tarragona,0,1,...,"Calma, vent molt suau",CapDeSetmana,2119,Tarda,Col.lisió d'un vehicle contra un obstacle de l...,dg,2018-09-30,21:19:00.000,21,19:00.000
16772,2018,Carretera,2018-04-04 10:53:00.000,SE,999999,VIVER I SERRATEIX,Bergueda,Barcelona,0,1,...,"Calma, vent molt suau",Feiners,1053,Matí,Bolcada a la calcada,dill-dij,2018-04-04,10:53:00.000,10,53:00.000


In [67]:
df_hor=df[["hores"]]
# num d'accidents en funció de la comarca i ordenats alfabèticament respecte del nom de la comarca
accperhor = pd.value_counts(df_hor['hores'])
accperhor = accperhor.to_frame() # convertir a Data Frame
accperhor.index.name = 'foo'
accperhor = accperhor.sort_values("foo") # la columna de velocitat es diu index, ordenar-la
accperhor = accperhor.rename(columns={"hores": "# accidents"}) # canviar els noms correctes
accperhor.head()

,# accidents
foo,
00,226
01,188
02,172
03,156
04,167


In [68]:
p_bar=accperhor.plot_bokeh.bar(ylabel='# Accidents', xlabel='Hora', title="Distr d'accidents per hora",alpha=0.6)

In [69]:
# Volem ara un boxplot, amb valor mitjà el num d'acc per hora fent mitjana aritmetica dividint 
# tots els nombres que apareixen en #accidents pel nombre de dies que abarca la recollida de dades
from datetime import date

d0 = date(2010, 1, 1)
d1 = date(2018, 12, 31)
delta = d1 - d0
N=delta.days
N

3286

In [84]:
accperhor["mitj"]=accperhor["# accidents"]/N
accperhor["desv"]=((1/N)*(accperhor["# accidents"]*(1-accperhor["mitj"])**2)+(N-accperhor["# accidents"])*accperhor["mitj"]**2)**0.5
accperhor

,# accidents,mitj,desv
foo,,,
00,226,0.07,3.81
01,188,0.06,3.19
02,172,0.05,2.93
03,156,0.05,2.66
04,167,0.05,2.85
05,305,0.09,5.08
06,428,0.13,6.97
07,676,0.21,10.52
08,793,0.24,12.06


In [ ]:
# Ara volem fer el box plot


In [ ]:
# Ara agruparem per intervals de pk de carreteres
# Fem un subdataset dels morts que n'hi ha en els accidents mortals que han ocorregut només en l'AP-7
dist_morts_via=df[['via','pk','nomcom','f_morts']]

#dist_morts_via=dist_morts_via.drop(index=dist_morts_via[dist_morts_via['f_morts']== '0'].index)
dist_morts_via.set_index('via', inplace=True)
dist_morts_ap=dist_morts_via.loc[['AP-7']]
dist_morts_ap["pk"]=dist_morts_ap["pk"].astype(int)
dist_morts_ap

In [ ]:
# Saber màxim pk
pks=dist_morts_ap[["pk"]]
max_val=pks.max()
max_val

In [ ]:
# Ordenem per punt km
dist_morts_ap=dist_morts_ap.sort_values('pk')
dist_morts_ap.head(40)

In [ ]:
# La taula anterior demostra que els punts km estan malament assignats, però anem a intentar tirar amb lo q
# tenim per veure si podem agrupar 
dist=dist_morts_ap.groupby(pd.cut(dist_morts_ap["pk"], np.arange(0, 3427+34.27, 34.27))).sum()
dist

In [ ]:
# Veiem que el que fa es agrupar els pks en intervals de 30 i sumar-los però nosaltres volem q ens digui quants sumands
# conformen eixa suma, quants pks està sumant (i.e. el nombre d'accidents en eixe interval), no el resultat d'aquesta suma

dist2=dist_morts_ap.groupby(pd.cut(dist_morts_ap["pk"], np.arange(0, 3427+34.27, 34.27))).size()
dist2=dist2.to_frame()
dist2=dist2.reset_index()
dist2=dist2.rename(columns={"pk": "Intervals pk", 0: "#accidents"}) # canviar els noms correctes
dist2['percentcarretera'] = dist2.index+1
pd.options.display.float_format = "{:,.2f}".format
dist2

In [ ]:
# Provem a fer un scatter

dist2.plot_bokeh(kind='scatter',x='percentcarretera',y='#accidents',title="Accidents en l'AP-7")

In [ ]:
plt.rcParams["figure.figsize"] = 10,4

x = dist2["percentcarretera"]
y = dist2["#accidents"]

fig, (ax,ax2) = plt.subplots(nrows=2, sharex=True)

extent = [0,100,0,1]
ax.imshow(y[np.newaxis,:], cmap="Greys", aspect="auto",extent=extent)
ax.set_yticks([])

ax2.plot(x,y)

plt.tight_layout()
plt.show()

In [ ]:
carr=df[["via","pk","nomcom","f_morts"]]
carr=carr.drop(index=carr[carr['via']== 'SE'].index)
#carr.set_index("via", inplace=True)
carr

In [ ]:
carrnomes=carr[["via"]]
p=carrnomes.value_counts().plot_bokeh(kind='pie')

In [ ]:
dist_acc_via=df[['via','pk','nomcom','f_morts']]

dist_acc_via=dist_acc_via.drop(index=dist_acc_via[dist_acc_via['f_morts']== '0'].index)
dist_acc_via.set_index('via', inplace=True)
dist_acc_n=dist_acc_via.loc[['N-II']]
dist_acc_n["pk"]=dist_acc_n["pk"].astype(int)
dist_acc_n